In [1]:
import ipywidgets as widgets 
import sqlite3
import re 
import urllib.request 
from urllib.parse import urlencode 
import json 
import csv 
from IPython.display import Image 
import pprint

#connection to database 
connect = sqlite3.connect('aa.db')

#Code for widgets 
#tab 1: select academy awards
style = {'description_width': 'initial'}
sel_aa = widgets.BoundedIntText(value=92, min=1, max=92,step=1,style=style,description='Select Academy Awards')
ta = widgets.Textarea()
all_awards = widgets.VBox([sel_aa, ta])

#tab 2: type an artist to see what movies they've done/demographics
sel_actor= widgets.Text(value='',placeholder='Type Actor/Actress Name',style=style,description='Actor/Actress Name')
actor_award=widgets.RadioButtons(options=['Won','Lost','Either'],value='Either',description='Select')
ta2 = widgets.Textarea()
all_actor = widgets.VBox([sel_actor, actor_award, ta2])

#tab 3: best actor/actress more than 2 times 
sel_cat = widgets.Dropdown(options=['Best Actor', 'Best Actress'],value='Best Actor',style=style,description='Academy Awards Categories')
ta3 = widgets.Textarea()
all_cat = widgets.VBox([sel_cat, ta3])

#code for tab widget 
children = [all_awards, all_actor, all_cat]
tab = widgets.Tab()
tab.children = children 
tab.set_title(0, 'Oscars Ceremony')
tab.set_title(1, 'Actor or Actress Name')
tab.set_title(2, "Multi-time Winners") 

#dashboard titles code 
html1 = "<h1 style= 'font-size: 20px'> Welcome Film Ethusiasts Far and Wide! </h1>"
html2 = "<h1 style= 'font-size: 20px'> 🏆Explore this dashboard to find out more about the Academy Awards, better known as the Oscars!🏆</h1>"
html3 = "<h1 style= 'font-size: 50px'> Former Academy Awards Data </h1>"
html4 = "<h1 style= 'font-size: 20px'> Interact with the 'Oscars Ceremony' tab to learn all about the winning films and actors for a given ceremony year.</h1>"
html5 = "<h1 style= 'font-size: 20px'> Interact with the 'Actor or Actress Name' tab to learn all about an actor's Academy wins, nominations, and demographics (if available).</h1>"
html6 = "<h1 style= 'font-size: 20px'> Interact with the 'Multi-time Winners' tab to learn all about an the 'Best' actors who have won more than one Academy award throughout their career.</h1>"

welcome = widgets.HTML(value=html1)
intro = widgets.HTML(value=html2)
header = widgets.HTML(value=html3)
tab1 = widgets.HTML(value=html4)
tab2 = widgets.HTML(value=html5)
tab3 = widgets.HTML(value=html6)
caption = widgets.Label(value='')
widgetlist = [welcome, intro, header, tab1, tab2, tab3, tab, caption]

dash = widgets.VBox(widgetlist)

observe_out = widgets.Output(layout={'border': '2.5px solid black', 'width': '700px'})
       
#CODE FOR TYPE IN ACTOR TAB  
actor_list = []
def actor_db(actor_name):
    actor_dem_query = f'''
    SELECT person, race_ethnicity, religion, sexual_orientation, birthplace, date_of_birth
    FROM demographics  
    WHERE year_of_award > 1929
    AND year_of_award < 2015
    AND person LIKE ?
    '''
    actor_dem_result = connect.execute(actor_dem_query, ('%' + actor_name +'%',)).fetchone()
    actor_list.clear()
    
    actor_dict = {}
    if actor_dem_result is not None: 
        actor_dict['name'] = actor_name
        actor_dict['race'] = actor_dem_result[1]
        actor_dict['religion'] = actor_dem_result[2]
        actor_dict['sex_orient'] = actor_dem_result[3]
        actor_dict['birthplace'] = actor_dem_result[4]
        actor_dict['birthday'] = actor_dem_result[5]
    else: 
        actor_dict['name'] = None
        actor_dict['race'] = None
        actor_dict['religion'] = None
        actor_dict['sex_orient'] = None
        actor_dict['birthplace'] = None
        actor_dict['birthday'] = None
        
    wl_sel = actor_award.value
    if wl_sel == 'Either': 
        actor_wins_query = f'''
        SELECT a.winner, a.film, a.category, a.year_ceremony, a.name
        FROM the_oscar_award a
        WHERE a.year_ceremony > 1929
        AND a.year_ceremony < 2015
        AND a.name like ? 
        GROUP BY a.film
        ORDER BY a.year_ceremony DESC
        '''
            
    elif wl_sel == 'Won':
        actor_wins_query = f''' 
        SELECT a.winner, a.film, a.category, a.year_ceremony, a.name
        FROM the_oscar_award a
        WHERE a.year_ceremony > 1929
        AND a.year_ceremony < 2015
        AND a.name like ? 
        AND a.winner = "True"
        GROUP BY a.film
        ORDER BY a.year_ceremony DESC
        '''
        
    else: 
        actor_wins_query = f''' 
        SELECT a.winner, a.film, a.category, a.year_ceremony, a.name
        FROM the_oscar_award a
        WHERE a.year_ceremony > 1929
        AND a.year_ceremony < 2015
        AND a.name like ? 
        AND a.winner = "False"
        GROUP BY a.film
        ORDER BY a.year_ceremony DESC
        '''
      
    actor_wl_result = connect.execute(actor_wins_query, ('%' + actor_name +'%',))  
    
    actor_results = []
    for row in actor_wl_result:
        info_dict = {}
        info_dict['won'] = row[0]
        info_dict['film'] = row[1]
        info_dict['cat'] = row[2]
        info_dict['year'] = row[3]
        actor_results.append(info_dict)
    
    actor_dict['movies'] = actor_results
    
    return actor_dict     
        

def actor_handler(change):
    actor_db(change['new'])
    observe_out.clear_output()
    #count = 0
    with observe_out:
        ta2.value = ''
        actor_dict = actor_db(sel_actor.value)

        name = actor_dict['name']
        race = actor_dict['race']
        religion = actor_dict["religion"]
        sex_orient = actor_dict["sex_orient"]
        birthplace = actor_dict["birthplace"]
        birthday = actor_dict["birthday"]
        movies = actor_dict['movies']

        movie_str = ''
        for i in range(len(movies)):
            movie = movies[i]
            won = movie['won']
            film = movie['film']
            cat = movie['cat']
            year = movie['year']
            movie_str += str(i +1) + '. ' + 'Film Name: ' + str(film) + ', ' + str(year) + '\n' + 'Award Category: ' + str(cat) + '\n' + 'Award Won? ' + str(won) + '\n' + '\n'

        wl_sel = actor_award.value
        if wl_sel == 'Won': 
            ta2.value += "Name: " + str(name) + '\n' + '\n' + "Personal Info and Demographics" + '\n' "Birthday: " + str(birthday) + '\n' + "Place of Birth: " + str(birthplace) + '\n' + "Race/Ethnicity: " + str(race) + '\n' + "Religion: " + str(religion) + '\n' + "Sexual Orientation: " + str(sex_orient) + '\n' + '\n' + "Performances" + '\n' + movie_str + '\n'
        else: 
            ta2.value += "Performances" + '\n' + movie_str + '\n'  
            
    caption.value = f'Currrent search: {sel_actor.value}'

def actor_win_handler(change):
    actor_db(sel_actor.value)
    observe_out.clear_output()
    #count = 0
    with observe_out:
        ta2.value = ''
        actor_dict = actor_db(sel_actor.value)

        name = actor_dict['name']
        race = actor_dict['race']
        religion = actor_dict["religion"]
        sex_orient = actor_dict["sex_orient"]
        birthplace = actor_dict["birthplace"]
        birthday = actor_dict["birthday"]
        movies = actor_dict['movies']

        movie_str = ''
        for i in range(len(movies)):
            movie = movies[i]
            won = movie['won']
            film = movie['film']
            cat = movie['cat']
            year = movie['year']
            movie_str += str(i +1) + '. ' + 'Film Name: ' + str(film) + ', ' + str(year) + '\n' + 'Award Category: ' + str(cat) + '\n' + 'Award Won? ' + str(won) + '\n' + '\n'

        wl_sel = actor_award.value
        if wl_sel == 'Won': 
            ta2.value += "Name: " + str(name) + '\n' + '\n' + "Personal Info and Demographics" + '\n' "Birthday: " + str(birthday) + '\n' + "Place of Birth: " + str(birthplace) + '\n' + "Race/Ethnicity: " + str(race) + '\n' + "Religion: " + str(religion) + '\n' + "Sexual Orientation: " + str(sex_orient) + '\n' + '\n' + "Performances" + '\n' + movie_str + '\n'
        else: 
            ta2.value += "Performances" + '\n' + movie_str + '\n'  
            
    caption.value = f'Currrent search: {sel_actor.value}'
    
#Tab 2: observe 
sel_actor.observe(actor_handler, names='value')  
actor_award.observe(actor_win_handler, names='value') 

#CORRECT CODE FOR CATEGORY TAB
def oscar_winner_sum(man_or_women):
    if man_or_women == 'Best Actor':
        man_or_women = 'Best actor'
    elif man_or_women == 'Best Actress':
        man_or_women = 'Best actress'
        
    result = connect.execute('''
    SELECT name, movie, oscar_yr, oscar_no
    FROM oscars
    WHERE award = ?
    ''',(man_or_women,))


    list_of_dict = []
    actors_added = []
    for actor in result:
        actor_dict = {}
        if actor[0] in actors_added:
            for dictionary in list_of_dict:
                if dictionary['name'] == actor[0]:
                    dictionary['movies'].append((actor[1],actor[2]))
                    dictionary['num_won']+=1
        else:
            actors_added.append(actor[0])
            actor_dict['name'] = actor[0]
            actor_dict['movies'] = [(actor[1],actor[2])]
            actor_dict['num_won']=1
            list_of_dict.append(actor_dict)

    return list_of_dict


#CORRECT CODE FOR CATEGORY TAB handler  
def multi_time_winner_handler(change):
    oscar_winner_sum(change['new'])
    observe_out.clear_output()
    with observe_out:
        caption.value = f"{change['new']} is selected"
        ta3.value = ''
        for i in oscar_winner_sum(sel_cat.value):
            name = i['name']
            num_won = i['num_won']
            movies = i['movies']
            movie_str = ''
            for i in range(len(movies)):
                movie = movies[i]
                movie_str += str(i +1) + '. ' + movie[0] + ', ' + '(' + str(movie[1]) + ')' + '\n'

            if num_won > 1:
                ta3.value += "Name: " + name + '\n' + "Number of Awards: " + str(num_won) + '\n' + "For Performances In: " + '\n' + movie_str + '\n'  
        
#observe 
sel_cat.observe(multi_time_winner_handler, names='value')

#oscar num tab 1
def oscarsNumber(number):
    oscar_num_query = f'''
    SELECT ceremony, year_ceremony, category, film, name
    FROM the_oscar_award
    WHERE winner = "True"
    AND ceremony = ?
    ORDER BY ceremony
    '''
    
    oscar_num_result = connect.execute(oscar_num_query, (number,))
    list_of_awards = []
    
    for oscar_cer in oscar_num_result:
        ceremony = oscar_cer[0]
        year = oscar_cer[1]
        award_dict = {}
        award_dict['award'] = oscar_cer[2]
        award_dict['film'] = oscar_cer[3]
        award_dict['person'] = oscar_cer[4]
        list_of_awards.append(award_dict)

    return (ceremony, year, list_of_awards)

def choose_aa_handler(change):
    oscarsNumber(change['new'])
    observe_out.clear_output()
    caption.value = f"Academy Award {change['new']} is selected."
    with observe_out:
        ta.value = ''
    (ceremony, cer_year, list_of_dict_of_award) = oscarsNumber(sel_aa.value)
    print_string = f'For Ceremony: {ceremony} In Year: {cer_year}\n\n'
    for award_i in list_of_dict_of_award:
        award = award_i['award']
        movie = award_i['film']
        person = award_i['person']
        string_to_add = f'Award Category: {award}\nFor Film: {movie}\nWith: {person}\n\n'
        print_string += string_to_add
    ta.value = print_string

#observe 
sel_aa.observe(choose_aa_handler, names = 'value')

#output                                   
display(dash, observe_out)



Output(layout=Layout(border='2.5px solid black', width='700px'))